In [5]:
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
import xgboost as xgb
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
from scipy.stats import randint
from sklearn.impute import SimpleImputer, KNNImputer
import pandas as pd
import numpy as np
import seaborn as sns
import statsmodels.api as sm
from lussi.stackoverflow import *
from lussi.ziprecruiter import *
nogit_data_dir = "622data_nogit"
wide_stack = load_stack(data_dir = nogit_data_dir, stack_type=StackType.WIDE)

In [4]:
# What does the data look like?
from itables import show
show(wide_stack.sample(n=10, random_state=42), scrollY='300px', paging=True)

In [8]:
# which cloumns are used when?filtered_columns = wide_stack.loc[:, 'Year':'AgeAvg']
filtered_columns = wide_stack.loc[:, 'Year':'AgeAvg']
# Group and calculate the percentage of non-null values for the filtered columns
grouped_summary_filtered = filtered_columns.groupby("Year").agg(lambda x: (x.notnull().mean() * 100)).reset_index()
# Round the result to one decimal place
grouped_summary_filtered_rounded = grouped_summary_filtered.round(1)
# Display the grouped summary
from IPython.display import display
display(grouped_summary_filtered_rounded)

,Year,OrgSize,Country,Employment,Gender,EdLevel,US_State,Age,DevType,Sexuality,Ethnicity,DatabaseWorkedWith,LanguageWorkedWith,PlatformWorkedWith,YearsCodePro,AnnualSalary,YearsCodeProAvg,OrgSizeAvg,AgeAvg
0,2017,75.7,100.0,100.0,68.2,100.0,0.0,0.0,70.3,0.0,64.3,57.3,71.3,56.7,99.5,25.1,96.6,72.7,0.0
1,2018,72.4,99.6,96.4,65.2,95.8,0.0,65.3,93.2,60.5,58.1,67.0,79.2,66.8,94.9,48.3,94.9,72.4,65.3
2,2019,80.8,99.9,98.1,96.1,97.2,0.0,89.1,91.5,85.7,86.3,85.5,98.5,90.8,83.6,0.0,83.6,75.9,89.1
3,2020,68.8,99.4,99.1,78.4,89.1,0.0,70.5,76.6,68.2,71.3,76.8,89.0,83.5,71.9,0.0,71.9,65.4,70.5
4,2021,72.8,100.0,99.9,98.6,99.6,17.9,98.8,79.7,87.9,95.2,83.3,98.7,62.5,73.4,56.1,73.4,66.3,98.1
5,2022,69.7,98.0,97.9,96.7,97.7,0.0,96.8,83.7,90.9,94.8,82.1,96.9,68.1,70.7,52.0,70.7,64.5,96.1
6,2023,72.9,98.6,98.6,0.0,98.6,0.0,100.0,86.2,0.0,0.0,82.3,97.7,71.3,74.2,53.8,74.2,66.8,99.5


In [9]:
# we're trying to predict Annual Salary. It's not in 2019 and 2020, so might as well just drop those, and then we'll look again.
wide_stack = wide_stack.query("Year != 2019 and Year != 2020")
filtered_columns = wide_stack.loc[:, 'Year':'AgeAvg']
# Group and calculate the percentage of non-null values for the filtered columns
grouped_summary_filtered = filtered_columns.groupby("Year").agg(lambda x: (x.notnull().mean() * 100)).reset_index()
# Round the result to one decimal place
grouped_summary_filtered_rounded = grouped_summary_filtered.round(1)
# Display the grouped summary
from IPython.display import display
display(grouped_summary_filtered_rounded)

,Year,OrgSize,Country,Employment,Gender,EdLevel,US_State,Age,DevType,Sexuality,Ethnicity,DatabaseWorkedWith,LanguageWorkedWith,PlatformWorkedWith,YearsCodePro,AnnualSalary,YearsCodeProAvg,OrgSizeAvg,AgeAvg
0,2017,75.7,100.0,100.0,68.2,100.0,0.0,0.0,70.3,0.0,64.3,57.3,71.3,56.7,99.5,25.1,96.6,72.7,0.0
1,2018,72.4,99.6,96.4,65.2,95.8,0.0,65.3,93.2,60.5,58.1,67.0,79.2,66.8,94.9,48.3,94.9,72.4,65.3
2,2021,72.8,100.0,99.9,98.6,99.6,17.9,98.8,79.7,87.9,95.2,83.3,98.7,62.5,73.4,56.1,73.4,66.3,98.1
3,2022,69.7,98.0,97.9,96.7,97.7,0.0,96.8,83.7,90.9,94.8,82.1,96.9,68.1,70.7,52.0,70.7,64.5,96.1
4,2023,72.9,98.6,98.6,0.0,98.6,0.0,100.0,86.2,0.0,0.0,82.3,97.7,71.3,74.2,53.8,74.2,66.8,99.5


In [14]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

# Step 1: Identify categorical columns and fill missing values if necessary
df = df.fillna('Unknown')  # Handle missing values (if any)

# Step 2: Apply one-hot encoding to categorical columns
df_encoded = pd.get_dummies(df, drop_first=True)

# Separate features and target
X = df_encoded.drop(['AnnualSalary', 'Year'], axis=1)  # Drop target and non-relevant columns
y = df_encoded['AnnualSalary']

# Step 3: Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 4: Train the RandomForestRegressor model
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Step 5: Get Feature Importances
importances = rf_model.feature_importances_
features = X.columns

# Step 6: Plot the top 15 features by importance
import matplotlib.pyplot as plt

sorted_indices = importances.argsort()[::-1]  # Sort features by importance

plt.figure(figsize=(10, 6))
plt.barh(features[sorted_indices[:15]], importances[sorted_indices[:15]])
plt.xlabel('Feature Importance')
plt.title('Top Feature Importance')
plt.gca().invert_yaxis()  # Flip y-axis for descending order
plt.show()

KeyError: "['AnnualSalary'] not found in axis"

In [ ]:
cols_to_keep = ['AnnualSalary','YearsCodeProAvg', 'OrgSizeAvg', 
                'scala', 'python', 'javascript', 'EdLevel',
                'postgresql', 'ethnicity_grouped', 'AgeAvg', 
               'gender_grouped','aws','mysql','redis','sql',
               'java','ruby','php','c','mongodb',
               'oracle','ibmdb2','sqlite','kubernetes',
               'linux','windows']
